# Lecture2

### 001 Array Seq

In [40]:
# Dynamic Array Seq
class Array_Seq:
    def __init__(self):
        self.A = []
        self.size = 0

    def __len__(self):
        return self.size

    def __iter__(self):
        yield from self.A

    def build(self, X):
        self.A = [a for a in X]
        self.size = len(self.A)
    
    def get_at(self, i):
        return self.A[i]
    
    def set_at(self, i, x):
        self.A[i] = x

    def _copy_forward(self, i, n, A, j):
        for k in range(n):
            A[j + k] = self.A[i + k]
    
    def _copy_backward(self, i, n, A, j):
        for k in range(n - 1, -1, -1):
            A[j + k] = self.A[i + k]
    
    def insert_at(self, i, x):
        n = len(self)
        A = [None] * (n + 1)
        self._copy_forward(0, i, A, 0)
        A[i] = x
        self._copy_forward(i, n - i, A, i + 1)
        self.build(A)
    
    def delete_at(self, i):
        n = len(self)
        A = [None] * (n - 1)
        self._copy_forward(0, i, A, 0)
        x = self.A[i]
        self._copy_forward(i + 1, n - i - 1, A, i)
        self.build(A)
    
    def insert_first(self, x):
        self.insert_at(0, x)

    def delete_first(self):
        self.delete_at(0)

    def insert_last(self, x):
        self.insert_at(len(self), x)
    
    def delete_last(self):
        self.delete_at(len(self) - 1)
    
    def __str__(self):
        return str(self.A)
    
    def __repr__(self):
        return str(self.A)
    

### 002 Dynamic Array Seq

In [4]:
class Dynamic_Array_Seq(Array_Seq):
    def __init__(self, r = 2):
        super().__init__()
        self.size = 0
        self.r = r
        self._compute_bounds()
        self._resize(0)
    
    def _compute_bounds(self):
        self.upper = len(self.A)
        self.lower = len(self.A) // (self.r * self.r)
    
    def _resize(self, n):
        if (self.lower < n < self.upper):
            return
        m = max(n, 1) * self.r
        A = [None] * m
        self._copy_forward(0, self.size, A, 0)
        self.A = A
        self._compute_bounds()
    
    def __len__(self):
        return self.size
    
    def __iter__(self):
        for i in range(len(self)):
            yield self.A[i]

    def insert_last(self, x):
        self._resize(self.size + 1)
        self.A[self.size] = x
        self.size += 1
    
    def build(self, X):
        for a in X:
            self.insert_last(a)
    
    def delete_last(self):
        self.A[self.size - 1] = None
        self.size -= 1
        self._resize(self.size)

    def insert_at(self, i, x):
        self.insert_last(None)
        self._copy_backward(i, self.size - i - 1, self.A, i + 1)
        self.A[i] = x
    
    def delete_at(self, i):
        x = self.A[i]
        self._copy_forward(i + 1, self.size - i - 1, self.A, i)
        self.delete_last()
    
    def insert_first(self, x):
        return self.insert_at(0, x)
    
    def delete_first(self):
        return self.delete_at(0)

In [ ]:
class BookMarkers(Array_Seq):
    def __init__(self):
        super().__init__()
        self.n_mark = 0
        self.mark = {'A': None, 'B': None}
    
    def place_mark(self, i, m):
        assert self.mark[m] is None, 'you can only shift marks'
        if self.n_mark == 0:
            A = [None] * (2 * self.size)
            self._copy_forward(0, i + 1, A, 0)
            self._copy_forward(i + 1, self.size - i - 1, A, self.size + i + 1)
            self.A = A
            self.mark[m] = (i, i + self.size + 1)
            self.n_mark += 1
            return
        if self.n_mark == 1:
            A = [None] * (3 * self.size)
            if m == 'A':
                assert self.mark['B'][0] > i, 'A must be before B'
                self._copy_forward(0, i + 1, A, 0)
                self._copy_forward(i + 1, 2 * self.size - i - 1, A, self.size + i + 1)
                self.A = A
                self.mark[m] = (i, i + self.size + 1)
                self.mark['B'] = (self.mark['B'][0] + self.size, self.mark['B'][1] + self.size)
                self.n_mark += 1
                return
            if m == 'B':
                assert self.mark['A'][0] < i, 'B must be after A'
                self._copy_forward(0, self.size + i + 1, A, 0)
                self._copy_forward(self.size + i + 1,  self.size - i - 1, A, 2 * self.size + i + 1)
                self.A = A
                self.mark[m] = (i + self.size, i + 2 * self.size + 1)
                self.n_mark += 1
                return

    def read_page(self, i):
        if self.n_mark == 0:
            return self.A[i]
        if self.n_mark == 1:
            marker = self.mark['A'] or self.mark['B']
            if i <= marker[0]:
                return self.A[i]
            else:
                return self.A[self.size + i]
        if self.n_mark == 2:
            markerA = self.mark['A'][0]
            markerB = self.mark['B'][0] - self.size
            if i <= markerA:
                return self.A[i]
            elif i <= markerB:
                return self.A[self.size + i]
            else:
                return self.A[2 * self.size + i]
    
    def shift_mark(self, m, d):
        assert self.mark[m] is not None, 'mark not found'

        left, right = self.mark[m]
        if d == 1:
            self.A[left + 1], self.A[right] = self.A[right], self.A[left + 1]
            self.mark[m] = (left + 1, right + 1)
        if d == -1:
            self.A[left], self.A[right - 1] = self.A[right - 1], self.A[left]
            self.mark[m] = (left - 1, right - 1)


    def move_page(self, m):
        assert self.n_mark == 2, 'must have two marks'
        n = 'A' if m == 'B' else 'B'
        left_m, right_m = self.mark[m]
        left_n, right_n = self.mark[n]
        assert right_n - left_n > 1
        self.A[left_m], self.A[left_n + 1] = self.A[left_n + 1], self.A[left_m]
        self.mark[m] = (left_m - 1, right_m)
        self.mark[n] = (left_n + 1, right_n)





bm = BookMarkers()
bm.build([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
bm.place_mark(5, 'B')
bm.place_mark(2, 'A')


print(bm)
for _ in range(3):
    bm.move_page('B')   
print(bm)

### 003 Linked List Seq

In [57]:
class Linked_List_Node:
    def __init__(self, x):
        self.item = x
        self.next = None
    
    def later_node(self, i):
        if i == 0:
            return self
        assert self.next
        return self.next.later_node(i - 1)

class Linked_List_Seq:
    def __init__(self):
        self.head = None
        self.size = 0

    def __len__(self):
        return self.size
    
    def __iter__(self):
        node = self.head
        while node:
            yield node.item
            node = node.next
    
    def insert_first(self, x):
        node = Linked_List_Node(x)
        node.next = self.head
        self.head = node
        self.size += 1
    
    def build(self, X):
        for a in reversed(X):
            self.insert_first(a)

    def get_at(self, i):
        node = self.head.later_node(i)
        return node.item
    
    def set_at(self, i, x):
        node = self.head.later_node(i)
        node.item = x

    def delete_first(self):
        x = self.head.item
        self.head = self.head.next
        self.size -= 1
        return x
    
    def insert_at(self, i, x):
        if i == 0:
            return self.insert_first(x)
        prev = self.head.later_node(i - 1)
        node = Linked_List_Node(x)
        node.next = prev.next
        prev.next = node
        self.size += 1
    
    def delete_at(self, i):
        if i == 0:
            return self.delete_first()
        node = self.head.later_node(i - 1)
        x = node.next.item
        node.next = node.next.next
        self.size -= 1
        return x
    
    def insert_last(self, x):
        self.insert_at(len(self), x)
    
    def delete_last(self):
        return self.delete_at(len(self) - 1)

    def __str__(self):
        return "[" + ", ".join(str(x) for x in self) + "]"



In [76]:
def reorder_students(L): 
    assert len(L) % 2 == 0
    previous = L.head.later_node(len(L) // 2 - 1)
    current = previous.next
    post = current.next
    while post.next is not None:
        # print(previous.item, current.item, post.item)
        post = current.next
        current.next = previous
        previous, current = current, post
    # print(previous.item, current.item)
    current.next = previous
    half = L.head.later_node(len(L) // 2 - 1)
    half.next.next = None
    half.next = current

ls = Linked_List_Seq()
ls.build([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
reorder_students(ls)
print(ls)

[1, 2, 3, 4, 5, 10, 9, 8, 7, 6]


In [82]:
def reverse(D, i, k):
    begin, end = i, i + k - 1
    while begin < end:
        x = D.delete_at(begin)
        D.insert_at(end, x)
        x = D.delete_at(end - 1)
        D.insert_at(begin, x)
        begin, end = begin + 1, end - 1
ls = Linked_List_Seq()
ls.build([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
reverse(ls, 2, 5)
print(ls)

[1, 2, 7, 6, 5, 4, 3, 8, 9, 10]


In [128]:
def move(D, i, k, j):
    for idx in range(k):
        if i > j:
            i_idx, j_idx = i + idx, j + idx
        else:
            i_idx, j_idx = i, j - 1
        x = D.delete_at(i_idx)
        D.insert_at(j_idx, x)
ls = Linked_List_Seq()
ls.build([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

move(ls, 7, 3, 2)
print(ls)

[1, 2, 8, 9, 10, 3, 4, 5, 6, 7]


In [159]:

class Doubly_Linked_List_Node:
    def __init__(self, x):
        self.key = x
        self.prev = None
        self.next = None

    def later_node(self, i):
        if i == 0: return self
        assert self.next
        return self.next.later_node(i - 1)

    def __repr__(self):
        return str(self.key)
    
    def __str__(self):
        return str(self.key)

class Doubly_Linked_List_Seq:
    def __init__(self):
        self.head = None
        self.tail = None

    def __iter__(self):
        node = self.head
        while node:
            yield node.key
            node = node.next


    def build(self, X):
        for a in X:
            self.insert_last(a)

    def get_at(self, i):
        node = self.head.later_node(i)
        return node.key

    def set_at(self, i, x):
        node = self.head.later_node(i)
        node.key = x

    def insert_first(self, x):
        ###########################
        # Part (a): Implement me! #
        ###########################
        node = Doubly_Linked_List_Node(x)
        if self.head is None:
            self.head = node
            self.tail = node
        else:
            node.next = self.head
            self.head.prev = node
            self.head = node
        return node

    def insert_last(self, x):
        ###########################
        # Part (a): Implement me! #
        ###########################
        node = Doubly_Linked_List_Node(x)
        if self.tail is None:
            self.tail = node
            self.head = node
        else:
            node.prev = self.tail
            self.tail.next = node
            self.tail = node
        return node
    def delete_first(self):

        ###########################
        # Part (a): Implement me! #
        ###########################
        assert self.head is not None
        x = self.head.key
        self.head = self.head.next
        if self.head is None:
            self.tail = None
        else:
            self.head.prev = None
        return x

    def delete_last(self):
        ###########################
        # Part (a): Implement me! #
        ###########################
        assert self.tail is not None
        x = self.tail.key
        self.tail = self.tail.prev
        if self.tail is None:
            self.head = None
        else:
            self.tail.next = None
        return x
    
    def remove(self, x1):
        if x1 == self.head:
            self.head = x1.next
        else:
            x1.prev.next = x1.next
        if x1 == self.tail:
            self.tail = x1.prev
        else:
            x1.next.prev = x1.prev
        return x1
        

    def remove2(self, x1, x2):

        ###########################
        # Part (b): Implement me! # 
        ###########################
        L2 = Doubly_Linked_List_Seq()
        L2.head, L2.tail = x1, x2
        if x1 == self.head:
            self.head = x2.next
        else:
            x1.prev.next = x2.next
            x1.prev = None
        if x2 == self.tail:
            self.tail = x1.prev
        else:
            x2.next.prev = x1.prev
            x2.next = None
        return L2

    def splice(self, x, L2):
        ###########################
        # Part (c): Implement me! # 
        ###########################
        xn = x.next
        x1, x2 = L2.head, L2.tail
        x.next = x1
        x1.prev = x
        x2.next = xn
        if xn is not None:
            xn.prev = x2
        else:
            self.tail = x2


# Lecture 3

In [13]:
# Simu a Set from Seq
def Set_from_Seq(seq):
    class set_from_seq:
        def __init__(self):
            self.S = seq()
        def __len__(self):
            return len(self.S)
        def __iter__(self):
            yield from self.S
        def build(self, A):
            self.S.build(A)
        def insert(self, x):
            for i in range(len(self.S)):
                if self.S.get_at(i).key == x.key:
                    self.S.set_at(i, x)
                    return
            self.S.insert_last(x)
        def delete(self, k):
            for i in range(len(self.S)):
                if self.S.get_at(i).key == k:
                    return self.S.delete_at(i)
        def find(self, k):
            for x in self:
                if x.key == k:
                    return x
            return None
        
        def findd_min(self):
            out = None
            for x in self:
                if out is None or x.key < out.key:
                    out = x
            return out
        
        def find_max(self):
            out = None
            for x in self:
                if out is None or x.key > out.key:
                    out = x
            return out
        
        def find_next(self, k):
            out = None
            for x in self:
                if x.key > k:
                    if (out is None) or (x.key < out.key):
                        out = x
            return out
        
        def find_prev(self, k):
            out = None
            for x in self:
                if x.key < k:
                    if (out is None) or (x.key > out.key):
                        out = x
            return out
        
        def iter_order(self):
            x = self.find_min()
            while x is not None:
                yield x
                x = self.find_next(x.key)
    return set_from_seq


In [144]:

class Sorted_Array_Set:
    def __init__(self):
        self.A = Array_Seq()
    def __len__(self):
        return len(self.A)
    def __iter__(self):
        yield from self.A
    
    def __str__(self):
        return str(self.A)
    
    def __repr__(self):
        return str(self.A)
    def iter_order(self):
        yield from self
    
    def build(self, X):
        self.A.build(X)
        self._sort()
    
    def _sort(self):
        self._merge_sort(self.A)
    
    def _merge_sort(self, A, a = 0, b = None):
        b = b or len(A)
        if b - a > 1:
            c = (a + b + 1) // 2
            self._merge_sort(A, a, c)
            self._merge_sort(A, c, b)
            L, R, i, j = [None] * (c - a), [None] * (b - c), 0, 0
            self.A._copy_forward(a, c - a, L, 0)
            self.A._copy_forward(c, b - c, R, 0)
            La, Ra = Array_Seq(), Array_Seq()
            La.build(L)
            Ra.build(R)
            while a < b:
                if j >= len(R) or (i < len(L) and La.get_at(i).key <= Ra.get_at(j).key):
                    A.set_at(a, La.get_at(i))
                    i += 1
                else:
                    A.set_at(a, Ra.get_at(j))
                    j += 1
                a += 1

    def _binary_search(self, k, i, j):
        if i >= j:
            return i
        m = (i + j) // 2
        x = self.A.get_at(m)
        if x.key > k:
            return self._binary_search(k, i, m - 1)
        if x.key < k:
            return self._binary_search(k, m + 1, j)
        return m
    
    def find_min(self):
        if len(self) == 0:
            return None
        return self.A.get_at(0)

    def find_max(self):
        if len(self) == 0:
            return None
        return self.A.get_at(len(self) - 1)
    
    def find(self, k):
        if len(self) == 0:
            return None
        i = self._binary_search(k, 0, len(self) - 1)
        x = self.A.get_at(i)
        return x if x.key == k else None

    def find_next(self, k):
        if len(self) == 0:
            return None
        i = self._binary_search(k, 0, len(self) - 1)
        x = self.A.get_at(i)
        if x.key > k:
            return x
        if i + 1 < len(self):
            return self.A.get_at(i + 1)
        return None
    
    def find_prev(self, k):
        if len(self) == 0:
            return None
        i = self._binary_search(k, 0, len(self) - 1)
        x = self.A.get_at(i)
        if x.key < k:
            return x
        if i - 1 >= 0:
            return self.A.get_at(i - 1)
        return None
    
    def insert(self, x):
        if len(self) == 0:
            self.A.insert_last(x)
            return
        i = self._binary_search(x.key, 0, len(self) - 1)
        k = self.A.get_at(i).key
        if k == x.key:
            self.A.set_at(i, x)
            return False
        if k > x.key:
            self.A.insert_at(i, x)
        else:
            self.A.insert_at(i + 1, x)
        return True
     
    def delete(self, k):
        i = self._binary_search(k, 0, len(self) - 1)
        assert self.A.get_at(i).key == k
        return self.A.delete_at(i)

In [7]:
# selection sort
def selection_sort(A, i = None):
    # O(n^2)
    if i is None:
        i = len(A) - 1
    if i > 0:
        j = prefix_max(A, i)
        A[i], A[j] = A[j], A[i]
        selection_sort(A, i - 1)

def prefix_max(A, i):
    # O(n)
    if i > 0:
        j = prefix_max(A, i - 1)
        if A[j] > A[i]:
            return j
    return i


[1, 1, 2, 3, 3, 4, 5, 5, 5, 6, 9]


In [18]:
def selection_sort(A):
    for i in range(len(A) - 1, 0, -1):
        m = i
        for j in range(i):
            if A[j] > A[m]:
                m = j
        A[i], A[m] = A[m], A[i]
# selection_sort([3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5])
pi = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5]
selection_sort(pi)
print(pi)

[1, 1, 2, 3, 3, 4, 5, 5, 5, 6, 9]


In [9]:
def insertion_sort(A, i = None):
    if i is None:
        i = len(A) - 1
    if i > 0:
        insertion_sort(A, i - 1)
        insert_last(A, i)

def insert_last(A, i):
    if i > 0 and A[i] < A[i-1]:
        A[i], A[i-1] = A[i-1], A[i]
        insert_last(A, i - 1)


[1, 1, 2, 3, 3, 4, 5, 5, 5, 6, 9]


In [19]:
def insertion_sort(A):
    for i in range(1, len(A)):
        j = i
        while j > 0 and A[j] < A[j - 1]:
            A[j], A[j - 1] = A[j - 1], A[j]
            j -= 1
pi = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5]
insertion_sort(pi)
print(pi)


[1, 1, 2, 3, 3, 4, 5, 5, 5, 6, 9]


In [11]:
def merge(L, R, A, i, j, a, b):
    if a < b:
        if (j <= 0) or (i > 0 and L[i-1] > R[j-1]):
            A[b-1] = L[i-1]
            i = i - 1
        else:
            A[b-1] = R[j-1]
            j = j - 1
        merge(L, R, A, i, j, a, b-1)

def merge_sort(A, a = 0, b = None):
    b = b or len(A)
    if 1 < b - a:
        c = (a + b + 1) // 2
        merge_sort(A, a, c)
        merge_sort(A, c, b)
        L, R = A[a:c], A[c:b]
        merge(L, R, A, len(L), len(R), a, b)


[1, 1, 2, 3, 3, 4, 5, 5, 5, 6, 9]


In [21]:
def merge_sort(A, a = 0, b = None):
    b = b or len(A)
    if 1 < b - a:
        c = (a + b + 1) // 2
        merge_sort(A, a, c)
        merge_sort(A, c, b)
        L, R = A[a:c], A[c:b]
        i, j = 0, 0
        for k in range(a, b):
            if j >= len(R) or (i < len(L) and L[i] <= R[j]):
                A[k] = L[i]
                i += 1
            else:
                A[k] = R[j]
                j += 1
pi = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5]
merge_sort(pi)
print(pi)   

[1, 1, 2, 3, 3, 4, 5, 5, 5, 6, 9]


### Direct access array

In [35]:
class DirectAccessArray:
    def __init__(self, u):
        self.A = [None] * u
    def find(self, k):
        return self.A[k]
    def insert(self, x):
        self.A[x.key] = x
    def delete(self, k):
        self.A[k] = None
    def find_next(self, k):
        for i in range(k, len(self.A)):
            if self.A[i] is not None:
                return self.A[i]
    def find_max(self):
        for i in range(len(self.A)-1, -1, -1):
            if self.A[i] is not None:
                return self.A[i]
    def delete_max(self):
        for i in range(len(self.A)-1, -1, -1):
            if self.A[i] is not None:
                x = self.A[i]
                self.A[i] = None
                return x

6

In [25]:
import random
class Hash_Table_Set:
    def __init__(self, r = 200):
        self.chain_set = Set_from_Seq(Linked_List_Seq)
        self.A = []  # 用于存储的chai数量m
        self.size = 0  # 存储的item数量n
        self.r = r
        self.p = 2**31 - 1
        self.a = random.randint(1, self.p - 1)
        self._compute_bounds()
        self._resize(0)

    def __len__(self):
        return self.size
    
    def __iter__(self):
        for X in self.A:
            yield from X

    def _hash(self, k, m):
        return ((self.a * k) % self.p) % m
    
    def _compute_bounds(self):
        self.upper = len(self.A)
        self.lower = len(self.A) * 100 * 100 // (self.r * self.r)
    
    def _resize(self, n):
        if not self.lower < n < self.upper:
            f = (self.r - 1) // 100 + 1
            m = max(n, 1) * f
            A = [self.chain_set() for _ in range(m)]
            for x in self:
                h = self._hash(x.key, m)
                A[h].insert(x)
            self.A = A
            self._compute_bounds()
    
    def build(self, X):
        for x in X:
            self.insert(x)

    def insert(self, x):
        self._resize(self.size + 1)
        h = self._hash(x.key, len(self.A))
        added = self.A[h].insert(x)
        if added:
            self.size += 1
        return added
    
    def find(self, k):
        h = self._hash(k, len(self.A))
        return self.A[h].find(k)
    
    def delete(self, k):
        assert len(self)>0
        h = self._hash(k, len(self.A))
        x = self.A[h].delete(k)
        self.size -= 1
        self._resize(self.size)
        return x
    
    def find_min(self):
        out = None
        for x in self:
            if (out is None) or (x.key < out.key):
                out = x
        return out
    
    def find_max(self):
        out = None
        for x in self:
            if (out is None) or (x.key> out.key):
                out = x
        return out
    
    def find_next(self, k):
        out = None
        for x in self:
            if x.key > k:
                if (out is None) or (x.key < out.key):
                    out = x
        return out
    
    def find_prev(self, k):
        out = None
        for x in self:
            if x.key < k:
                if (out is None) or (x.key > out.key):
                    out = x
        return out
    
    def iter_order(self):
        x = self.find_min()
        while x is not None:
            yield x
            x = self.find_next(x.key)
    
    
                
    

In [36]:
def dup_brute_force(A):
    for i in range(len(A) - 1):
        for j in range(i + 1, len(A)):
            if A[i] == A[j]:
                return True
    return False

def dup_sort(A):
    merge_sort(A)
    cur = None
    for i in A:
        if cur is not None and i == cur:
            return True
        cur = i
    return False


A = [2,2,  3, 4, 5, 6, 9, 8]
dup_sort(A)

True

In [110]:

class PhotoShop:
    def __init__(self):
        self.id_seq = Sorted_Array_Set()
        self.docu = Doubly_Linked_List_Seq()
    
    def import_image(self, x):
        photo = self.docu.insert_last(x)
        self.id_seq.insert(photo)
    
    def display(self):
        return list(self.docu)
        
    def move_below(self, x, y):
        px = self.id_seq.find(x)
        py = self.id_seq.find(y)
        assert px and py and px != py
        
        if px.prev:
            px.prev.next = px.next
        else:
            self.docu.head = px.next
        if px.next:
            px.next.prev = px.prev
        else:
            self.docu.tail = px.prev
        
        px.next = py
        px.prev = py.prev

        if py.prev:
            py.prev.next = px
        else:
            self.docu.head = px
        py.prev = px

pt = PhotoShop()
pt.import_image(3)
for i in range(5, 10):
    pt.import_image(i)
pt.import_image(4)
pt.move_below(3, 4)
pt.display()

[5, 6, 7, 8, 9, 3, 4]

In [129]:
def merge(L, R, A, i, j, a, b):
    if a < b:
        if (j <= 0) or (i > 0 and L[i-1] > R[j-1]):
            A[b-1] = L[i-1]
            i = i - 1
        else:
            A[b-1] = R[j-1]
            j = j - 1
        merge(L, R, A, i, j, a, b-1)

def merge_sort(A, a = 0, b = None):
    b = b or len(A)
    if 1 < b - a:
        c = (a + b + 1) // 2
        merge_sort(A, a, c)
        merge_sort(A, c, b)
        L, R = A[a:c], A[c:b]
        merge(L, R, A, len(L), len(R), a, b)

def new_merge_sort(H):
    D = [1 for _ in H]
    H2 = [(H[i], i) for i in range(len(H))]
    def sort(A, a = 0, b = None):
        b = b or len(A)
        if 1 < b - a:
            c = (a + b + 1) // 2
            sort(A, a, c)
            sort(A, c, b)
            i, j, L, R = 0, 0, A[a:c], A[c:b]
            while a < b:
                if (j>=len(R)) or (i < len(L) and L[i][0] <= R[j][0]):
                    D[L[i][1]] += j

                    A[a] = L[i]
                    i += 1
                else:
                    A[a] = R[j]
                    j += 1
                a += 1
    sort(H2)
    return D

H = [34, 57, 70, 19, 48, 2, 94, 7, 63, 75]
new_merge_sort(H)

[4, 5, 6, 3, 3, 1, 4, 1, 1, 1]

In [178]:
## problem set2 q4
class Viewers:
    def __init__(self, v):
        self.key = v
        self.messages = Linked_List_Seq()
    def send_message(self, m):
        self.messages.insert_first(m)
    def __repr__(self):
        return f"Viewers({self.key}, {list(self.messages)})"

class TubeChat:
    def __init__(self, V):
        self.viewers = Sorted_Array_Set()
        self.viewers.build([Viewers(v) for v in V])
        self.messages = Doubly_Linked_List_Seq()
    
    def send_message(self, v, m):
        viewer = self.viewers.find(v)
        assert viewer is not None
        message = self.messages.insert_first(m)
        viewer.send_message(message)
    
    def __iter__(self):
        cur = self.messages.head
        while cur is not None:
            yield cur.key
            cur = cur.next

    def recent(self, k):
        return list(self)[:k]
    
    def ban(self, v):
        viewer = self.viewers.find(v)
        assert viewer is not None
        while len(viewer.messages) > 0:
            cur = viewer.messages.delete_first()
            self.messages.remove(cur)


V = [3, 5, 7, 9]
tc = TubeChat(V)
tc.send_message(5, "hello")
tc.send_message(3, "hi")
tc.send_message(7, "good morning")
tc.send_message(5, "how are you?")
tc.ban(3)



TypeError: remove() missing 1 required positional argument: 'x2'